In [3]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapipe as mp
from pathlib import Path  # for safer path joins if needed
import pandas as pd

# access utils if using shared code
import sys
sys.path.append("..")

In [4]:
def roi_crop(input_path, output_path, label):

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True, 
    refine_landmarks=True, 
    max_num_faces=1, 
    min_detection_confidence=0.5 # inc if too many fase detections
)
    LEFT_EYE_LANDMARKS = [463, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362]
    RIGHT_EYE_LANDMARKS = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]
    LEFT_IRIS_LANDMARKS = [474, 475, 477, 476]
    RIGHT_IRIS_LANDMARKS = [469, 470, 471, 472]

    for file in os.listdir(input_path):
        img_path = os.path.join(input_path, file)

        try:
            # loads the image
            img = cv2.imread(img_path)

            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                raise Exception
            
            # convert img to RGB for mediapipe
            # opencv default is BGR
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            results = face_mesh.process(img_rgb)

            if not results.multi_face_landmarks:
                print(f"No face detected: {file}")
                continue
            
            # get actual positions on img
            h, w, _ = img.shape
            landmarks = results.multi_face_landmarks[0]
            eye_points = []
            for idx in LEFT_EYE_LANDMARKS + RIGHT_EYE_LANDMARKS:
                x = int(landmarks.landmark[idx].x * w)  # scale x to image width
                y = int(landmarks.landmark[idx].y * h)  # scale y to image height
                eye_points.append((x, y))
            
            x_coords, y_coords = zip(*eye_points)  # separates xs and ys

            # find min and max x and y for bounding rectangle
            x_min, x_max = max(min(x_coords), 0), min(max(x_coords), w)
            y_min, y_max = max(min(y_coords), 0), min(max(y_coords), h)

            roi = img[y_min:y_max, x_min:x_max]  # Crop using the box
            roi_resized = cv2.resize(roi, (224, 224))  # Resize to model input size

        except Exception as e:
            print(f"Error processing {file}: {e}")